In [87]:
pwd

'C:\\Users\\Aakash Sadaphule\\Desktop\\Untitled Folder'

In [88]:
import json
import pandas as pd
import numpy as np
from pandas import DataFrame

In [89]:
#reading the dataset
dataset_path = 'C:/Users/Aakash Sadaphule/Desktop/Untitled Folder/archive/MMHS150K_GT.json'
with open(dataset_path, 'r') as f:
  hs_data = json.loads(f.read())

In [90]:
with open ("C:/Users/Aakash Sadaphule/Desktop/Untitled Folder/archive/MMHS150K_GT.json") as f:
    data = json.load(f)

In [91]:
data=pd.DataFrame.from_dict(data)
data=data.T
data=data.reset_index()
data.rename(columns={"index":"user_id"},inplace=True)

In [92]:
print('No. of rows: {} and columns are: {}'.format(data.shape[0],data.shape[1]))
data.head()

No. of rows: 149823 and columns are: 6


,user_id,img_url,labels,tweet_url,tweet_text,labels_str
0,1114679353714016256,http://pbs.twimg.com/tweet_video_thumb/D3gi9MH...,"[4, 1, 3]",https://twitter.com/user/status/11146793537140...,@FriskDontMiss Nigga https://t.co/cAsaLWEpue,"[Religion, Racist, Homophobe]"
1,1063020048816660480,http://pbs.twimg.com/ext_tw_video_thumb/106301...,"[5, 5, 5]",https://twitter.com/user/status/10630200488166...,My horses are retarded https://t.co/HYhqc6d5WN,"[OtherHate, OtherHate, OtherHate]"
2,1108927368075374593,http://pbs.twimg.com/media/D2OzhzHUwAADQjd.jpg,"[0, 0, 0]",https://twitter.com/user/status/11089273680753...,“NIGGA ON MA MOMMA YOUNGBOY BE SPITTING REAL S...,"[NotHate, NotHate, NotHate]"
3,1114558534635618305,http://pbs.twimg.com/ext_tw_video_thumb/111401...,"[1, 0, 0]",https://twitter.com/user/status/11145585346356...,RT xxSuGVNGxx: I ran into this HOLY NIGGA TODA...,"[Racist, NotHate, NotHate]"
4,1035252480215592966,http://pbs.twimg.com/media/Dl30pGIU8AAVGxO.jpg,"[1, 0, 1]",https://twitter.com/user/status/10352524802155...,“EVERYbody calling you Nigger now!” https://t....,"[Racist, NotHate, Racist]"


In [93]:
import nltk
from nltk import tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')
nltk.download('rslp')
nltk.download('punkt')
import re

[nltk_data] Downloading package stopwords to C:\Users\Aakash
[nltk_data]     Sadaphule\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to C:\Users\Aakash
[nltk_data]     Sadaphule\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Aakash
[nltk_data]     Sadaphule\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Data preporcessing 

In [94]:
def preprocessing(data):
    stemmer = nltk.stem.RSLPStemmer()
    stop_words = stopwords.words('english')
    stop_words.remove('not')
    text_data = []
    for tweet in data:
      review = re.sub(r"@[A-Za-z0-9_]+", " ", tweet)
      review = re.sub('RT', ' ', review)
      review = re.sub(r"https?://[A-Za-z0-9./]+", " ", review)
      review = re.sub(r"https?", " ", review)
      review = re.sub('[^a-zA-Z]', ' ', review)
      review = review.lower()
      review = review.split()
      porter_stemmer = PorterStemmer()
      review = [porter_stemmer.stem(word) for word in review if not word in set(stop_words) if len(word) > 2]
      review = ' '.join(review)
      text_data.append(review)

    return np.array(text_data)

In [95]:
text_data = preprocessing(data['tweet_text'])
text_data

array(['nigga', 'hors retard',
       'nigga momma youngboy spit real shit nigga', ...,
       'nigga big shitti',
       'say nigga rich amp said anger mmph lhhni lhhnyreunion',
       'nigga joe budden said thano got galact ass'], dtype='<U110')

In [96]:
Nd = data.copy()
Nd['labels'].value_counts
new_label = []
for x in Nd['labels'].values:
    new_label.append(x[0])
    
Nd['new_label'] = new_label
Nd['new_label']

0         4
1         5
2         0
3         1
4         1
         ..
149818    2
149819    0
149820    0
149821    1
149822    0
Name: new_label, Length: 149823, dtype: int64

# TF-IDF

In [99]:
label = lambda labels_list: [0 if x == 0 else 1 for x in labels_list]
flattened = [[key, val['tweet_text'], 1 if sum(to_label(val['labels'])) ==3 else 0] 
             for key, val in hs_data.items()]
flattened[:10]

[['1114679353714016256', '@FriskDontMiss Nigga https://t.co/cAsaLWEpue', 1],
 ['1063020048816660480', 'My horses are retarded https://t.co/HYhqc6d5WN', 1],
 ['1108927368075374593',
  '“NIGGA ON MA MOMMA YOUNGBOY BE SPITTING REAL SHIT NIGGA” https://t.co/UczofqHrLq',
  0],
 ['1114558534635618305',
  'RT xxSuGVNGxx: I ran into this HOLY NIGGA TODAY 😭😭😭😭 https://t.co/Wa6Spl9kIw',
  0],
 ['1035252480215592966',
  '“EVERYbody calling you Nigger now!” https://t.co/6mguJ6KIBF',
  0],
 ['1106978219654303744',
  '“ real ass bitch give a fuck boutta nigga” https://t.co/YtyXf52oGi',
  0],
 ['1113920043568463874',
  '@WhiteHouse @realDonaldTrump Fuck ice. White supremacist trash. All of you are racist garbage. https://t.co/o2OB4yUnex',
  1],
 ['1114588617693966336', 'Day’s a cunt https://t.co/Ie6QZReHsw', 0],
 ['1045809514740666370', '#sissy faggot https://t.co/bm1nk8HcYO', 0],
 ['1108178453910695936', '@Gloriko_ Nigga what? https://t.co/nOwIJtgtU1', 0]]

In [102]:
import numpy
HD = numpy.array(falttened)
HD = falttened
HD[:10]

['@FriskDontMiss Nigga https://t.co/cAsaLWEpue',
 'My horses are retarded https://t.co/HYhqc6d5WN',
 '“NIGGA ON MA MOMMA YOUNGBOY BE SPITTING REAL SHIT NIGGA” https://t.co/UczofqHrLq',
 'RT xxSuGVNGxx: I ran into this HOLY NIGGA TODAY 😭😭😭😭 https://t.co/Wa6Spl9kIw',
 '“EVERYbody calling you Nigger now!” https://t.co/6mguJ6KIBF',
 '“ real ass bitch give a fuck boutta nigga” https://t.co/YtyXf52oGi',
 '@WhiteHouse @realDonaldTrump Fuck ice. White supremacist trash. All of you are racist garbage. https://t.co/o2OB4yUnex',
 'Day’s a cunt https://t.co/Ie6QZReHsw',
 '#sissy faggot https://t.co/bm1nk8HcYO',
 '@Gloriko_ Nigga what? https://t.co/nOwIJtgtU1']

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [103]:
def vectorizer(text_data) -> list:
    vectorizer = TfidfVectorizer(
        analyzer="word",
        tokenizer=None,
        preprocessor=None,
        stop_words=stopwords.words('english'),
        max_features=200
    )

    train_data_features = vectorizer.fit_transform(text_data)
    print(f'vocabulary: {vectorizer.vocabulary_}')
    print(f'vocabulary size is {len(vectorizer.vocabulary_)}')

    return train_data_features

In [137]:
vectorized = vectorizer(HD)
vectorized.shape

vocabulary: {'nigga': 116, 'https': 84, 'co': 26, 'retarded': 137, 'real': 132, 'shit': 147, 'rt': 139, 'today': 165, 'calling': 23, 'nigger': 118, 'ass': 4, 'bitch': 14, 'give': 64, 'fuck': 54, 'realdonaldtrump': 133, 'white': 187, 'trash': 168, 'day': 34, 'cunt': 31, 'faggot': 47, 'twitter': 174, 'word': 191, 'fat': 48, 'getting': 63, 'another': 2, 'sjw': 149, 'wanna': 181, 'talk': 156, 'first': 51, 'time': 164, 'says': 144, 'never': 113, 'surrender': 154, 'really': 134, 'said': 141, 'dyke': 42, 'twat': 173, 'always': 0, 'bad': 8, 'still': 151, 'like': 91, 'called': 22, 'man': 105, 'see': 145, 'need': 112, 'little': 93, 'dumb': 41, 'niggas': 117, 'got': 72, 'play': 126, 'look': 97, 'happy': 76, 'big': 12, 'love': 100, 'ya': 195, 'im': 85, 'god': 66, 'damn': 33, 'take': 155, 'card': 24, 'retard': 136, 'ever': 44, 'stop': 152, 'know': 87, 'oh': 121, 'let': 89, 'go': 65, 'get': 61, 'amp': 1, 'new': 114, 'every': 45, 'beat': 9, 'bro': 18, 'fucking': 56, 'make': 104, 'people': 125, 'right

(149823, 200)

In [111]:
from sklearn.model_selection import train_test_split
X = vectorized
y = [row[2] for row in HD]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

In [142]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='sgd',
                    alpha=1e-5,
                    hidden_layer_sizes=(100,),
                    random_state=42)
clf.fit(X_train, y_train)

results = clf.predict(X_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [143]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, results)
print("Accuracy score is: %.4f\n" % accuracy)

Accuracy score is: 0.1653



# Word2Vec model

In [147]:
pip install -U gensim

Requirement already up-to-date: gensim in c:\programdata\anaconda3\lib\site-packages (4.0.1)
Note: you may need to restart the kernel to use updated packages.


In [148]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

In [149]:
hsVec = [nltk.word_tokenize(tweet_text) for tweet_text in text_data]

In [150]:
hsVec[:10]

[['nigga'],
 ['hors', 'retard'],
 ['nigga', 'momma', 'youngboy', 'spit', 'real', 'shit', 'nigga'],
 ['xxsugvngxx', 'ran', 'holi', 'nigga', 'today'],
 ['everybodi', 'call', 'nigger'],
 ['real', 'ass', 'bitch', 'give', 'fuck', 'boutta', 'nigga'],
 ['fuck', 'ice', 'white', 'supremacist', 'trash', 'racist', 'garbag'],
 ['day', 'cunt'],
 ['sissi', 'faggot'],
 ['nigga']]

In [178]:
#training the model
vec_model =gensim.models.Word2Vec(
    hsVec, 
    vector_size=100, 
    window=5, 
    min_count=1, 
    workers=4)
vec_model.save("word2vec.model")

In [186]:
# top 5 similar word to "nigga"
word="nigga"
vec_model.wv.most_similar (positive=word,topn=5)

[('hoe', 0.8540602326393127),
 ('dat', 0.8509646654129028),
 ('wit', 0.8476145267486572),
 ('cuz', 0.8444004058837891),
 ('imma', 0.8442403674125671)]

In [187]:
# top 5 similar word to "racist"
word2="racist"
vec_model.wv.most_similar (positive=word2,topn=5)

[('ignor', 0.9185754656791687),
 ('poor', 0.9153494834899902),
 ('support', 0.9071479439735413),
 ('nazi', 0.9054499268531799),
 ('inbr', 0.8917744755744934)]

In [170]:
vec_model = Word2Vec.load("word2vec.model")

In [172]:
import gensim.downloader
print(list(gensim.downloader.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [173]:
glove_vectors = gensim.downloader.load('glove-twitter-25')
glove_vectors.most_similar('nigga')

[('lil', 0.9541468620300293),
 ('bitch', 0.952037513256073),
 ('mf', 0.9506526589393616),
 ('bruh', 0.9500959515571594),
 ('dawg', 0.948303759098053),
 ('fuckin', 0.9452494382858276),
 ('shawty', 0.9435693025588989),
 ('ass', 0.9413026571273804),
 ('cuz', 0.9359691739082336),
 ('shit', 0.9340022802352905)]